In [1]:
from paperswithtopic.config import load_config
from paperswithtopic.preprocess import Preprocess
from paperswithtopic.dataloader import get_dataloader
from paperswithtopic.run import run, setup
from paperswithtopic.misc import seed_everything, get_today

from IPython.display import clear_output

C:\Users\pha\anaconda3\envs\xai501\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
cfg = load_config()
cfg.use_saved = True
cfg.pre_embed = False
cfg.use_bert_embed = False

In [3]:
cfg.PTH_DIR = os.path.join(cfg.PTH_DIR, f'{get_today()}_{cfg.model_name}/')

In [4]:
cfg.PTH_DIR

'./models//20210605-1739_bertclassification'

In [3]:
preprocess = Preprocess(cfg=cfg)

In [5]:
X, y = preprocess.pp_pipeline()
idx2word = preprocess.idx2word

[load_data] 0.1 sec 
There are 49980 papers.
[retrieve_raw_papers] 0.0 sec 
[remove_unknown] 0.3 sec 
[build_idx2word] 19.2 sec 
[tokenize_papers] 0.3 sec 
[pp_pipeline] 20.0 sec 


In [13]:
sample_paper = 'Adversarial attacks for computer vision'

In [5]:
from glob import glob

In [ ]:
LOAD_DIR = 